Cargamos el árchivo A1.4-Vino-Tinto.csv con la librería de pandas e imprimimos la dimensión de este como los primeros 5 filas de cada columna.

In [80]:
import pandas as ps
datos=ps.read_csv("A1.4-Vino-Tinto.csv")
print("Dimensiones:", datos.shape)
print(datos.head(5))

Dimensiones: (1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        5  
3        6

Aplicamos la función train_test_split de la librería sklearn.model_selection y quitamos la variable de calidad para x y en y la llamamos. A su vez ahora dividiremos con la función del inicio en 4 partes, siendo para X train y test al igual que para y train y test, dividiendolos en 80/20 e imprimimos cada dimensión de el entrenamiento y pruebas, corroborando que sea la misma dimensión de todo el archivo.

In [93]:
from sklearn.model_selection import train_test_split

X = train.drop('calidad', axis=1)
y= train.calidad

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

print("Entrenamiento:", train.shape)
print("Pruebas:",test.shape)
print(train.head(10))

Entrenamiento: (1279, 12)
Pruebas: (320, 12)
      acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
911          9.1           0.28          0.46             9.0     0.114   
8            7.8           0.58          0.02             2.0     0.073   
790          7.4           0.37          0.43             2.6     0.082   
885          8.9           0.75          0.14             2.5     0.086   
1324         6.7           0.46          0.24             1.7     0.077   
2            7.8           0.76          0.04             2.3     0.092   
725          9.0           0.66          0.17             3.0     0.077   
365         10.0           0.42          0.50             3.4     0.107   
807          8.2           0.28          0.40             2.4     0.052   
17           8.1           0.56          0.28             1.7     0.368   

      dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  \
911                  3.0                 9.0   0.99901 

Generamos la función SequentialFeatureSelector de la librería mlxtend.feature_selection abreviandolo a SFS y a su vez también importamos la función LinearRegression de la librería sklearn.linear_model  para poder ejecutar de manera correcta la metodología de selección hacia adelante.

Al linearRegression() le llamamos estimator para hacerlo mas visibles, a su vez ponemos los siguientes códigos para que funcione de la manera adecuada para estos datos:
- k_features de 2,8 para seleccionar la cantidad de columnas
- Forward true para que vaya hacia adelante 
- Scoring en 'r2' para que nos proporciones el R² de la aproximación lineal para saber que tan buena es
- Cv=10 para asi realizar validación cruzada, y cuántas instancias de la misma.

A su vez convertimos a este modelo a un fit de X_train y y_train y lo imprimimos.

In [94]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

estimator=LinearRegression()

sfsForward=SFS(estimator,
                  k_features=(2,8), 
                  forward=True,      
                  scoring='r2',     
                  cv=10)
modeloF=sfsForward.fit(X_train, y_train)
print("Variables seleccionadas:\n", modeloF.k_feature_names_)

Variables seleccionadas:
 ('acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


Generamos la función r2_score de la librería sklearn.metrics
Primero convertiremos el ModeloF a una lista para poder agrupar los datos de manera correcto, despuéscategorizaremos el X para forward tanro en train y en test en base a la lista que creamos.
Luego crearemos un modelos fit de la regresión lineal  y la y predicción para el forward usando el comando predictic. Para poder asi obtener el R² de forward con la funcion previamente llamada en conjunto del y test y forwards.

In [95]:
from sklearn.metrics import r2_score
sfs_variaF=list(modeloF.k_feature_names_)

X_trainForward=X_train[sfs_variaF]
X_testForward = X_test[sfs_variaF]

linearR.fit(X_trainForward, y_train)
y_predForward=linearR.predict(X_testForward)

r2Forward=r2_score(y_test, y_predForward)
print("R² del modelo Forward:", r2Forward)

R² del modelo Forward: 0.37047640740230625


Volvemos a aplicar la secuencia que en el punto 3, pero ahora definiendo los  datos para un Backward, para esto cambiamos el forward a false y el k_features a 2,5 para tener menos variables de inicio.

In [96]:
sfsBackward=SFS(estimator,
                  k_features=(2,5), 
                  forward=False,
                  scoring='r2',     
                  cv=10)
modeloB=sfsBackward.fit(X_train, y_train)
print("Variables seleccionadas:\n", modeloB.k_feature_names_)

Variables seleccionadas:
 ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')


Repetiremos el proceso del punto 4 ahora con las variables del Backward y el puntos 5, a su vez imprimiendo una conclusión a estos dos diferentes procesos.


In [97]:
sfs_variaB=list(modeloB.k_feature_names_)

X_trainBackward=X_train[sfs_variaB]
X_testBackward=X_test[sfs_variaB]

linearR.fit(X_trainBack, y_train)
y_predBackward=linearR.predict(X_testBackward)

r2Backward=r2_score(y_test, y_predBackward)
print("R² Backward:", r2Backward)

print("Considero que el Forward fue mejor en este caso, puesto que tiene un R² mas grande, aunque tenga mas variables, lo cual")
print("podría significar que si la dejamos en 5 variables pueda tener mejor precisión que el modelo Backward")

R² Backward: -0.010496017239536526
Considero que el Forward fue mejor en este caso, puesto que tiene un R² mas grande, aunque tenga mas variables, lo cual
podría significar que si la dejamos en 5 variables pueda tener mejor precisión que el modelo Backward
